In [29]:
from google.cloud import bigquery
import os
import zipfile
import polars as pl
import pandas as pd
from datetime import datetime

# Define the dataset and table names
dataset_id = 'wedge-project-JBangtson.the_wedge_dataset'

# Initialize a BigQuery client
client = bigquery.Client(project='wedge-project-JBangtson')


data_directory = "E:\\College\\Fall 2024\\ADA\\Wedge\\Wedge_Project\\data\\unzipped\\"

## Importing the files in as lazy dataframes.

In [23]:
# Need to put all csvs into there own dfs
#https://chatgpt.com/share/66e4ad8b-ea5c-8000-9117-d884dd0bbfb3


# Initialize an empty list to store LazyFrames
lazy_df_list = []

# Loop through files and load lazily
for idx, file in enumerate(os.listdir(data_directory)):
    

    file_path = os.path.join(data_directory, file)


    if len(os.listdir(data_directory)[idx].split("_")) >= 4 and os.listdir(data_directory)[idx].split("_")[3] == "inactive.csv":
        # Use LazyFrame for efficient processing
        lazy_df = pl.scan_csv(file_path, has_header=True, null_values=["\\N"], ignore_errors=True,separator=";")

    else:
        # Use LazyFrame for efficient processing
        lazy_df = pl.scan_csv(file_path, has_header=True, null_values=["\\N"], ignore_errors=True)
    
    # Append LazyFrame to the list
    lazy_df_list.append(lazy_df)

# Example: Materialize (collect) one of the lazy DataFrames to inspect it
df = lazy_df_list[0].collect()
#df1 = lazy_df_list[38].collect()

clean_columns = df.columns
#print(df.head())


## Creating GBQ Schema

In [22]:

schema = [
    bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),#
    bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("scale", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemQtty", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volSpecial", "FLOAT", mode="NULLABLE"),###
    bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memType", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("staff", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("charflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("batchHeaderID", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("display", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE"),
]

table_id = ""
table = bigquery.Table(table_id, schema=schema)

client.create_table(table)


ValueError: table_id must be a fully-qualified ID in standard SQL format, e.g., "project.dataset.table_id", got 

## Uploading to GBQ (TK 🔥)

In [35]:
# Date time
pandas_df = df.to_pandas()


# Fixing the code

pandas_df["datetime"] = pandas_df["datetime"].apply(datetime_safe_cast)
   

In [39]:
for col_name in pandas_df:
    print(f'Column: {col_name}')

    if col_name == "datetime":
        pandas_df[col_name] = pandas_df[col_name].apply(datetime_safe_cast)
        
    #print(col_data)

Column: datetime
Column: register_no
Column: emp_no
Column: trans_no
Column: upc
Column: description
Column: trans_type
Column: trans_subtype
Column: trans_status
Column: department
Column: quantity
Column: Scale
Column: cost
Column: unitPrice
Column: total
Column: regPrice
Column: altPrice
Column: tax
Column: taxexempt
Column: foodstamp
Column: wicable
Column: discount
Column: memDiscount
Column: discountable
Column: discounttype
Column: voided
Column: percentDiscount
Column: ItemQtty
Column: volDiscType
Column: volume
Column: VolSpecial
Column: mixMatch
Column: matched
Column: memType
Column: staff
Column: numflag
Column: itemstatus
Column: tenderstatus
Column: charflag
Column: varflag
Column: batchHeaderID
Column: local
Column: organic
Column: display
Column: receipt
Column: card_no
Column: store
Column: branch
Column: match_id
Column: trans_id


----
## Cleaning/Casting Methods

### Datetime

In [34]:
from datetime import datetime

# Date time

def datetime_safe_cast(val):
    try:
        return datetime.strptime(val, '%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        return None  # or another value to handle invalid cases

        

### Float

In [12]:
# Float
def float_safe_cast(val):
    try:
        return float(val)
    except ValueError:
        return None  # or another value to handle invalid cases




### String

In [13]:
# String
def string_safe_cast(val):
    try:
        return str(val)
    except ValueError:
        return ""  # or another value to handle invalid cases



### Boolean

In [14]:

# Boolean
def boolean_safe_cast(val):
    try:
        return bool(val)
    except ValueError:
        return None  # or another value to handle invalid cases